In [6]:
# Read in the data
# Data must be txt file to preserve the Time field (otherwise the csv uses scientific 
# notation, which python reads as a string)

import pandas as pd
import csv, os
DATA_DIR = '../../project_data/pipeline'
#DATA_FILE = 'dataall_sample_resultant.txt'
DATA_FILE = 'data.json'

#data = pd.read_csv(os.path.join(DATA_DIR, DATA_FILE), sep="\t")
data = pd.read_json(os.path.join(DATA_DIR, DATA_FILE), lines=True)
data.head()

Time  hea_Acc_X  hea_Acc_Y  hea_Acc_Z  hea_Acc_res  \
0  1305484680656250  -8.394493  -0.289654  -4.694934     9.622568   
1  1305484680664060  -8.424558  -0.269359  -4.715950     9.658463   
2  1305484680671870  -8.463954  -0.246909  -4.757963     9.712759   
3  1305484680679680  -8.484299  -0.205610  -4.802305     9.751294   
4  1305484680687500  -8.514286  -0.168922  -4.821060     9.785920   

   hea_Ang_Vel_X  hea_Ang_Vel_Y  hea_Ang_Vel_Z  hea_Ang_Vel_res  \
0       0.233155       0.108864      -0.009238         0.257484   
1       0.229700       0.123943      -0.021526         0.261892   
2       0.228067       0.125588      -0.021534         0.261248   
3       0.232877       0.125719      -0.027637         0.266084   
4       0.236284       0.127487      -0.039812         0.271419   

   hea_Mag_Fie_X       ...         wai_Acc_Z  wai_Acc_res  wai_Ang_Vel_X  \
0      37.393494       ...          4.564053     9.817315       0.133363   
1      38.592620       ...          4.568816     9.858511       0.133187   
2      37.964825       ...          4.566459     9.858342       0.139337   
3      37.338184       ...          4.568791     9.839856       0.140834   
4      38.592620       ...          4.575866     9.853872       0.133677   

   wai_Ang_Vel_Y  wai_Ang_Vel_Z  wai_Ang_Vel_res  wai_Mag_Fie_X  \
0       1.189985       0.025180         1.197700      33.129913   
1       1.185260      -0.028650         1.193064      35.989338   
2       1.192400      -0.010660         1.200561      28.483034   
3       1.181461      -0.010720         1.189874      33.175020   
4       1.188079       0.013213         1.195649      35.708830   

   wai_Mag_Fie_Y  wai_Mag_Fie_Z  wai_Mag_Fie_res  
0     -46.746370       2.408075        57.346429  
1     -38.425260      -4.238263        52.817571  
2     -44.536907      -4.562881        53.062597  
3     -46.936607      -0.022347        57.477191  
4     -43.920162       0.162534        56.605015  

[5 rows x 87 columns]

In [8]:
data.shape

(1190369, 87)

In [9]:
non_grouping_cols = ['Time', 'trial_id', 'label'] 
cols = non_grouping_cols + [col for col in data.columns.tolist() if col not in non_grouping_cols]
data = data[cols]
headers = list(data.columns)
grouping_indices = list(range(3,len(headers)))

In [11]:
test = data.values

In [64]:
import numpy as np

WINDOW_LENGTH = .1 #seconds

# Intialize data
windows = []
window = {headers[i]: [] for i in grouping_indices}
previous_window, previous_trial, previous_label = None,  test[0][1], test[0][2]
start_time = test[0][0]

# Create windows
count = 0
for row in test:
    # Check times to see if we are in same window still
    # If we are in the same window, just append values
    current_time, trial, label = row[0], row[1], row[2]
    time_diff = (current_time - start_time) / 1000000 # convert microseconds to seconds
    #print(trial, current_time, start_time, time_diff)
    # If we are in a new window or new trial, record values for previous window and start new one
    if time_diff > WINDOW_LENGTH or trial != previous_trial: 
        #print(trial, previous_trial, 'new')
        new_row = []
        for i in grouping_indices:
            values = window[headers[i]] # get the list of values for each sensor
            new_row += [max(values)]
        windows.append([start_time, previous_trial, previous_label] + new_row)
        previous_window, previous_trial, previous_label = window, trial, label
        window = {headers[i]: [] for i in grouping_indices}
        start_time = current_time
        
    for i in grouping_indices:
        window[headers[i]].append(row[i])

if len(window[headers[4]]) > 0:
    new_row = []
    for i in grouping_indices:
        values = window[headers[i]] # get the list of values for each sensor
        new_row += [max(values)]
    windows.append([start_time, previous_trial, previous_label] + new_row)

    
#df = pd.DataFrame(windows, columns=schema)
df = pd.DataFrame(windows, columns=cols)

In [67]:
# write to txt/csv
# change extension and uncomment lines to write as CSV instead of TXT
# need to write to TXT b/c the csv write the 'Time' in scientific notation

OUTPUT_DIR = '../../project_data/pipeline'
OUTPUT_NAME = 'data_windows_01s.txt'
# OUTPUT_NAME = 'test.csv'

df.to_csv(os.path.join(OUTPUT_DIR, OUTPUT_NAME), sep='\t', index=False)
#df.to_csv(os.path.join(OUTPUT_DIR, OUTPUT_NAME), index=False)

In [66]:
df[df['trial_id'] == 2].Time.tolist()#.head(40)

[1305484166195310.0,
 1305484166296870.0,
 1305484166398430.0,
 1305484166500000.0,
 1305484166601560.0,
 1305484166703120.0,
 1305484166804680.0,
 1305484166906250.0,
 1305484167007810.0,
 1305484167109370.0,
 1305484167210930.0,
 1305484167312500.0,
 1305484167414060.0,
 1305484167515620.0,
 1305484167617180.0,
 1305484167718750.0,
 1305484167820310.0,
 1305484167921870.0,
 1305484168023430.0,
 1305484168125000.0,
 1305484168226560.0,
 1305484168328120.0,
 1305484168429680.0,
 1305484168531250.0,
 1305484168632810.0,
 1305484168734370.0,
 1305484168835930.0,
 1305484168937500.0,
 1305484169039060.0,
 1305484169140620.0,
 1305484169242180.0,
 1305484169343750.0,
 1305484169445310.0,
 1305484169546870.0,
 1305484169648430.0,
 1305484169750000.0,
 1305484169851560.0,
 1305484169953120.0,
 1305484170054680.0,
 1305484170156250.0,
 1305484170257810.0,
 1305484170359370.0,
 1305484170460930.0,
 1305484170562500.0,
 1305484170664060.0,
 1305484170765620.0,
 1305484170867180.0,
 130548417096

1304798066828120.0